<a href="https://colab.research.google.com/github/aghapygad336/MENA/blob/master/gtd_happiness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.ensemble import ExtraTreesClassifier, AdaBoostRegressor
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import random
import pandas as pd
import numpy as np
from openpyxl.utils import dataframe
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
import seaborn as sns
from math import sqrt
import matplotlib.pyplot as plt
import math
from matplotlib import colors as cs
from scipy.stats import pearsonr
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import scale
from sklearn import preprocessing, metrics
from sklearn.feature_selection import SelectKBest
from sklearn import tree
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, precision_score, recall_score, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.model_selection import GridSearchCV

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**The Global Terrorism Database (GTD)** 
documents more than 190,000 international and domestic terrorist attacks that occurred worldwide since 1970. With details on various dimensions of each attack, the GTD familiarizes analysts, policymakers, scholars, and journalists with patterns of terrorism. The GTD defines terrorist attacks as: Acts by non-state actors involving the threatened or actual use of illegal force or violence to attain a political, economic, religious, or social goal through fear, coercion, or intimidation. Data collection is ongoing and updates are published annually at.

In [0]:
import pandas as pd
import numpy as np
data2015 = pd.read_csv('/content/drive/My Drive/MENA/2015.csv',encoding = "ISO-8859-1", engine = 'python')
data2016 = pd.read_csv('/content/drive/My Drive/MENA/2016.csv',encoding = "ISO-8859-1", engine = 'python')
data2017 = pd.read_csv('/content/drive/My Drive/MENA/2017.csv',encoding = "ISO-8859-1", engine = 'python')
happiness = data2015.append(data2016.append(data2017,sort=True),sort= True)
gtd = pd.read_csv('/content/drive/My Drive/MENA/golbal.csv',encoding = "ISO-8859-1", engine = 'python', usecols=[1,2,3,8,10,19,20,21,22,26,27,29,35,41,71,84,100,101,103,58])
gtd = gtd[(gtd['region_txt'] =='Middle East & North Africa') & (gtd['iyear'] > 2000)]


**Missing values** are representative of the messiness of real world data. There can be a multitude of reasons why they occur — ranging from human errors during data entry, incorrect sensor readings, to software bugs in the data processing pipeline.




**Categorial** Let’s start with the most simple thing you can do: removal. As mentioned before, while this is a quick solution, and might work in some cases when the proportion of missing values is relatively low (<10%), most of the time it will make you lose a ton of data. Imagine that just because of missing values in one of your features you have to drop the whole observation, even if the rest of the features are perfectly filled and informative!

**Numerical NaNs** A standard and often very good approach is to replace the missing values with mean, median or mode. For numerical values you should go with mean, and if there are some outliers try median (since it is much less sensitive to them).

In [0]:
# gtd missing values
numeric_ = gtd._get_numeric_data()
cat = gtd.select_dtypes(include='object')
mean_nan = numeric_.fillna(numeric_.mean()).dropna(axis=1, how='all')
cat_nan = cat.loc[:, cat.isnull().mean() <0.25]
gtd = mean_nan.join(cat_nan)


In [0]:
numeric_ = happiness._get_numeric_data()
cat = happiness.select_dtypes(include='object')
mean_nan = numeric_.fillna(numeric_.mean()).dropna(axis=1, how='all')
cat_nan = cat.loc[:, cat.isnull().mean() <0.25]
happiness = mean_nan.join(cat_nan)

Better encoding of categorical data can mean better model performance. In this series I’ll introduce you to a wide range of encoding options .


In [0]:
def encoding(dfData):    
    df = dfData.copy().select_dtypes(include='object')
    # print(df.columns)
    rows,cols = dfData.shape
    encodings = {}
    for col in df.columns:
        Region_nums = []
        Region_nums = df[col].unique()
        Region_nums = { Region_nums[i]: i for i in range(len(Region_nums)) }
        encodings.update({col:Region_nums})
        dfData[col].replace(Region_nums, inplace=True)
    return dfData,encodings

def get_by_value(dic,value):
    for key, val in dic.items():
        if val == value :
            return key    

In [0]:
gtd, gtd_encodings = encoding(gtd)
happiness, happiness_encodings = encoding(happiness)


Having about 600 terrorist group name (class labels), with some groups with less
than 10 attacks which may cause misleading results. You are required to reduce the
number of class labels to the 5 groups with max number of attacks

In [0]:
top_5 = gtd['gname'].value_counts()[1:6]
top_5 = [203,27,107,67,58]
gtd = gtd[gtd['gname'].isin(top_5)]


In [0]:

combinedData=pd.merge(gtd, happiness, left_on='country_txt', right_on='Country')
happiness, happiness_encodings = encoding(happiness)
combinedDataE, combinedData_encodings = encoding(combinedData)


In [0]:
def SplitTrainTes(dataSet):
    train_dataSet=dataSet[(dataSet['iyear'] >=2000) & (dataSet['iyear'] <= 2016)]
    test_dataSet=dataSet[dataSet['iyear'] ==2017]
    return train_dataSet,test_dataSet

**Training and Testing**

---


a. You are required to use the tuples of years from [2000. 2016] as training set
b. Tuples of year 2017 will be used as testing set.
c. We have 2 approaches in this assignment
i. Train and test using features of GTD only.
ii. Train and test using features of GTD and WHR combined.

In [0]:
train_GTD,test_GTD=SplitTrainTes(gtd)
train_Combined,test_Combined=SplitTrainTes(combinedData)

In [0]:
def splitX_SplitY(trainD,testD):
    x_train = trainD.drop("gname", axis=1)
    y_train = trainD["gname"]
    x_test = testD.drop("gname", axis=1)
    y_test = testD["gname"]
    return x_train,y_train,x_test,y_test

In [0]:
x_trainGTD,y_trainGTD,x_testGTD,y_testGTD=splitX_SplitY(train_GTD,test_GTD)
x_trainCombined,y_trainCombined,x_testCombined,y_testCombined=splitX_SplitY(train_Combined,test_Combined)


**The F measure** 

---
(F1 score or F score) is a measure of a test's accuracy of   **Classifier Models** and is defined as the weighted harmonic mean of the precision and recall of the test. 

In [0]:
from sklearn.metrics import multilabel_confusion_matrix
def Fmeasure(estimator,x_train,y_train,x_test,y_test):
    y_pred_train = estimator.predict(x_train)
    y_pred_test = estimator.predict(x_test)
    print('Train Set Accuracy : ', accuracy_score(y_train, y_pred_train))
    print('Train Set Precision : ', precision_score(y_train, y_pred_train, average='micro'))
    print('Train Set Recall : ', recall_score(y_train, y_pred_train, average='micro'))
    print('Train F-Score for each class : ', f1_score(y_train, y_pred_train, average='micro'))
    print('Train Mean F-Score for both classes : ', f1_score(y_train, y_pred_train, average='micro'))

    print('----------------------------------------------------------------------')
    print('Test Set Accuracy : ', accuracy_score(y_test, y_pred_test))
    print('Test Set Precision : ', precision_score(y_test, y_pred_test, average='micro'))
    print('Test Set Recall : ', recall_score(y_test, y_pred_test, average='micro'))
    print('Test F-Score for each class : ', f1_score(y_test, y_pred_test, average='micro'))
    print('Test Mean F-Score for both classes : ', f1_score(y_test, y_pred_test, average='micro'))

    print('----------------------------------------------------------------------')

In [0]:
def kNearestNeighborsFunction(x_train,y_train,x_test,y_test):
    neighbors = list(range(1, 30, 2))
    # empty list that will hold cv scores
    cv_scores = []
    # perform 10-fold cross validation
    for k in neighbors:
        knn = KNeighborsClassifier(n_neighbors=k)
        scores = cross_val_score(knn, x_train, y_train, cv=10, scoring='accuracy')
        cv_scores.append(scores.mean())

    mse = [1 - x for x in cv_scores]
    optimal_k = neighbors[mse.index(min(mse))]
    n=optimal_k
    print('Best N found at ' , n)
    knn = KNeighborsClassifier(n_neighbors=n)
    knn.fit(x_train,y_train)
    Fmeasure(knn,x_train,y_train,x_test,y_test)

In [49]:

kNearestNeighborsFunction(x_trainGTD,y_trainGTD,x_testGTD,y_testGTD)

Best N found at  3
Train Set Accuracy :  0.9203720219136196
Train Set Precision :  0.9203720219136196
Train Set Recall :  0.9203720219136196
Train F-Score for each class :  0.9203720219136196
Train Mean F-Score for both classes :  0.9203720219136196
----------------------------------------------------------------------
Test Set Accuracy :  0.8993329290479078
Test Set Precision :  0.8993329290479078
Test Set Recall :  0.8993329290479078
Test F-Score for each class :  0.8993329290479077
Test Mean F-Score for both classes :  0.8993329290479077
----------------------------------------------------------------------


In [0]:
kNearestNeighborsFunction(x_trainCombined,y_trainCombined,x_testCombined,y_testCombined)


In [0]:

def DT (x_train,y_train,x_test,y_test):
    ct_gini = DecisionTreeClassifier()
    ct_entropy = DecisionTreeClassifier(criterion="entropy")
    ct_gini = ct_gini.fit(x_train, y_train)
    ct_entropy = ct_entropy.fit(x_train, y_train)
    Fmeasure(ct_entropy,x_train,y_train,x_test,y_test)
    Fmeasure(ct_gini,x_train,y_train,x_test,y_test)

In [0]:
DT(x_trainGTD,y_trainGTD,x_testGTD,y_testGTD)

In [0]:
DT(x_trainCombined,y_trainCombined,x_testCombined,y_testCombined)


In [0]:
def NB(x_train,y_train,x_test,y_test):
    nb = GaussianNB()
    nb = nb.fit(x_train, y_train)
    Fmeasure(nb,x_train,y_train,x_test,y_test)


In [0]:
NB(x_trainGTD,y_trainGTD,x_testGTD,y_testGTD)

In [0]:
NB(x_trainCombined,y_trainCombined,x_testCombined,y_testCombined)


In [0]:
def LR(x_train,y_train,x_test,y_test):
    import warnings
    warnings.filterwarnings("ignore")

    grid = {"C": np.logspace(-3, 3, 7), "penalty": ["l1", "l2"]}  # l1 lasso l2 ridge
    logreg = LogisticRegression()
    logreg_cv = GridSearchCV(logreg, grid, cv=10)
    logreg_cv.fit(x_train, y_train)
    print("tuned hpyerparameters :(best parameters) ", logreg_cv.best_params_)
    Fmeasure(logreg_cv,x_train,y_train,x_test,y_test)
    print("accuracy :", logreg_cv.best_score_)

In [0]:
LR(x_trainGTD,y_trainGTD,x_testGTD,y_testGTD)

In [0]:
def AB(x_train,y_train,x_test,y_test):
    import warnings
    warnings.filterwarnings("ignore")

    param_grid = {
        'learning_rate': [.1, .2, .3, .4, .5],

        'n_estimators': [50, 100, 150, 200, 250]
    }

    classifier = AdaBoostClassifier()
    grid_Search = GridSearchCV(classifier, param_grid=param_grid)
    kk = grid_Search.fit(x_train, y_train)
    Fmeasure(kk,x_train,y_train,x_test,y_test)

In [0]:
AB(x_trainGTD,y_trainGTD,x_testGTD,y_testGTD)

In [0]:
AB(x_trainCombined,y_trainCombined,x_testCombined,y_testCombined)


In [0]:
def SVM(x_train,y_train,x_test,y_test):
    from sklearn.svm import SVC
    svclassifier = SVC(kernel='poly', degree=8)
    svclassifier.fit(x_train, y_train)
    Fmeasure(svclassifier,x_train,y_train,x_test,y_test)



In [0]:
SVM(x_trainGTD,y_trainGTD,x_testGTD,y_testGTD)